In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os

# Load the HAM10000 metadata
metadata = pd.read_csv('HAM10000_metadata.csv')

# Preprocess the data
labels = metadata['dx']  # 'dx' column contains the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
metadata['encoded_labels'] = encoded_labels

# Split the data into training, validation, and test sets
train_data, test_data = train_test_split(metadata, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Image data generator for data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    preprocessing_function=preprocess_input  # Use VGG19 preprocessing function
)

# Specify the path to the folder containing images
images_folder = 'E:/new project cs/Dataset'

# Load and preprocess the images for training, validation, and test sets
def load_and_preprocess(data):
    images = []
    for image_id in data['image_id']:
        img_path = os.path.join(images_folder, f'{image_id}.jpg')
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        images.append(img_array)
    
    images = np.array(images)
    labels = tf.keras.utils.to_categorical(data['encoded_labels'], num_classes=len(label_encoder.classes_))
    
    return images, labels

train_images, train_labels = load_and_preprocess(train_data)
val_images, val_labels = load_and_preprocess(val_data)
test_images, test_labels = load_and_preprocess(test_data)

# Load pre-trained VGG19 model without the top layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Spatial Attention Layer
class SpatialAttentionLayer(layers.Layer):
    def __init__(self):
        super(SpatialAttentionLayer, self).__init__()

    def build(self, input_shape):
        self.conv1 = layers.Conv2D(1, (1, 1), activation='sigmoid', padding='same', kernel_initializer='he_normal')
        super(SpatialAttentionLayer, self).build(input_shape)

    def call(self, x):
        avg_pool = tf.reduce_mean(x, axis=[3], keepdims=True)
        max_pool = tf.reduce_max(x, axis=[3], keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=3)
        attention = self.conv1(concat)
        return x * attention

# Build the model with attention mechanism
model = models.Sequential([
    base_model,
    SpatialAttentionLayer(),
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(datagen.flow(train_images, train_labels, batch_size=64),
                    epochs=20, validation_data=(val_images, val_labels))

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(datagen.flow(test_images, test_labels))
print(f"Test accuracy: {test_accuracy}")

# Save the model (optional)
model.save("skin_cancer_detection_model.h5")





Epoch 1/20


101/101 [==============================] - 1195s 12s/step - loss: 1.3750 - accuracy: 0.6670 - val_loss: 1.1418 - val_accuracy: 0.6606
Epoch 2/20
101/101 [==============================] - 1216s 12s/step - loss: 1.1235 - accuracy: 0.6723 - val_loss: 1.1218 - val_accuracy: 0.6606
Epoch 3/20
101/101 [==============================] - 1238s 12s/step - loss: 1.0240 - accuracy: 0.6842 - val_loss: 1.0713 - val_accuracy: 0.6625
Epoch 4/20
101/101 [==============================] - 1174s 12s/step - loss: 0.9298 - accuracy: 0.6946 - val_loss: 1.0280 - val_accuracy: 0.6744
Epoch 5/20
101/101 [==============================] - 1160s 11s/step - loss: 0.8297 - accuracy: 0.7145 - val_loss: 1.0496 - val_accuracy: 0.6812
Epoch 6/20
101/101 [==============================] - 1174s 12s/step - loss: 0.7799 - accuracy: 0.7285 - val_loss: 1.0254 - val_accuracy: 0.6794
Epoch 7/20
101/101 [==============================] - 1148s 11s/step - loss: 0.7219 - accuracy: 0.7461 - val_loss: 1.0244 - v

c:\Users\jerin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import os

# Define the custom layer
class SpatialAttentionLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(SpatialAttentionLayer, self).__init__()

    def build(self, input_shape):
        self.conv1 = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid', padding='same', kernel_initializer='he_normal')
        super(SpatialAttentionLayer, self).build(input_shape)

    def call(self, x):
        avg_pool = tf.reduce_mean(x, axis=[3], keepdims=True)
        max_pool = tf.reduce_max(x, axis=[3], keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=3)
        attention = self.conv1(concat)
        return x * attention

# Load the HAM10000 metadata
metadata = pd.read_csv('HAM10000_metadata.csv')

# Preprocess the data
labels = metadata['dx']  # 'dx' column contains the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
metadata['encoded_labels'] = encoded_labels

# Split the data into training, validation, and test sets
train_data, test_data = train_test_split(metadata, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Image data generator for data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    preprocessing_function=preprocess_input  # Use VGG19 preprocessing function
)

# Specify the path to the folder containing images
images_folder = 'E:/new project cs/Dataset'

# Load and preprocess the images for training, validation, and test sets
def load_and_preprocess(data):
    images = []
    for image_id in data['image_id']:
        img_path = os.path.join(images_folder, f'{image_id}.jpg')
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        images.append(img_array)
    
    images = np.array(images)
    labels = tf.keras.utils.to_categorical(data['encoded_labels'], num_classes=len(label_encoder.classes_))
    
    return images, labels

# Assuming you already have the test_data
test_images, test_labels = load_and_preprocess(test_data)

# Load the saved model with custom_objects
saved_model_path = "E:/new project cs/New folder/skin_cancer_detection_model.h5"
model = tf.keras.models.load_model(saved_model_path, custom_objects={'SpatialAttentionLayer': SpatialAttentionLayer})

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(datagen.flow(test_images, test_labels))
print(f"Test accuracy: {test_accuracy}")

# Obtain predictions on the test data
predictions = model.predict(datagen.flow(test_images, batch_size=64, shuffle=False))

# Convert predictions and true labels to class indices
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_labels, axis=1)

# Create and print the confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Create and print the classification report
class_report = classification_report(true_classes, predicted_classes)
print("\nClassification Report:")
print(class_report)



63/63 [==============================] - 281s 4s/step - loss: 0.6489 - accuracy: 0.7673
Test accuracy: 0.7673490047454834
32/32 [==============================] - 304s 9s/step
Confusion Matrix:
[[  29   14    9    0    3   13    1]
 [  16   47   11    0    6   12    1]
 [  15   10   96    0   30   77    0]
 [   4    1    5    1    0   16    1]
 [  12    5   28    1   79   99    2]
 [   6   19   21    0   19 1268    5]
 [   0    0    0    0    1    7   13]]

Classification Report:
              precision    recall  f1-score   support

           0       0.35      0.42      0.38        69
           1       0.49      0.51      0.50        93
           2       0.56      0.42      0.48       228
           3       0.50      0.04      0.07        28
           4       0.57      0.35      0.43       226
           5       0.85      0.95      0.90      1338
           6       0.57      0.62      0.59        21

    accuracy                           0.77      2003
   macro avg       0.56   

In [37]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os

# Load the HAM10000 metadata
metadata = pd.read_csv('HAM10000_metadata.csv')

# Preprocess the data
labels = metadata['dx']  # 'dx' column contains the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
metadata['encoded_labels'] = encoded_labels

# Define the custom layer (SpatialAttentionLayer)
class SpatialAttentionLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(SpatialAttentionLayer, self).__init__()

    def build(self, input_shape):
        self.conv1 = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid', padding='same', kernel_initializer='he_normal')
        super(SpatialAttentionLayer, self).build(input_shape)

    def call(self, x):
        avg_pool = tf.reduce_mean(x, axis=[3], keepdims=True)
        max_pool = tf.reduce_max(x, axis=[3], keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=3)
        attention = self.conv1(concat)
        return x * attention

# Load the saved model with custom layer
loaded_model = load_model("E:/new project cs/New folder/skin_cancer_detection_model.h5", custom_objects={'SpatialAttentionLayer': SpatialAttentionLayer})

# Specify the path to the image you want to predict
image_path = "E:/new project cs/New folder (2)/test/akiec/ISIC_0028158.jpg"

# Load and preprocess the image
img = image.load_img(image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Make predictions
predictions = loaded_model.predict(img_array)

# Get the predicted class label
predicted_class = np.argmax(predictions)

# Decode the class label using the label encoder
predicted_label = label_encoder.classes_[predicted_class]

print(f"Predicted class: {predicted_label}")


1/1 [==============================] - 1s 947ms/step
Predicted class: nv


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import os

# Load the HAM10000 metadata
metadata = pd.read_csv('HAM10000_metadata.csv')

# Preprocess the data
labels = metadata['dx']  # 'dx' column contains the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
metadata['encoded_labels'] = encoded_labels

# Define the custom layer (SpatialAttentionLayer)
class SpatialAttentionLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(SpatialAttentionLayer, self).__init__()

    def build(self, input_shape):
        self.conv1 = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid', padding='same', kernel_initializer='he_normal')
        super(SpatialAttentionLayer, self).build(input_shape)

    def call(self, x):
        avg_pool = tf.reduce_mean(x, axis=[3], keepdims=True)
        max_pool = tf.reduce_max(x, axis=[3], keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=3)
        attention = self.conv1(concat)
        return x * attention

# Load the saved model with custom layer
loaded_model = tf.keras.models.load_model("E:/new project cs/New folder/skin_cancer_detection_model.h5", custom_objects={'SpatialAttentionLayer': SpatialAttentionLayer})

# Specify the path to the image you want to predict
image_path = "E:/new project cs/New folder (2)/test/akiec/ISIC_0024450.jpg"

# Load and preprocess the image
img = image.load_img(image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Make predictions
predictions = loaded_model.predict(img_array)

# Assuming you have a dictionary mapping numerical labels to full class names
class_mapping = {
    0: 'atinic keratoses and intraepithelial carcinoma',
    1: 'basal cell carcinoma',
    2: 'benign keratosis-like lesions',
    3: 'dermatofibroma',
    4: 'melanoma',
    5: 'melanocytic nevi',
    6: 'vascular lesions',
}

# Get the predicted class index
predicted_class = np.argmax(predictions)

# Decode the class label using the label encoder
predicted_label = label_encoder.classes_[predicted_class]

# Use the mapping to get the full form of the predicted class
predicted_full_form = class_mapping.get(predicted_class, 'Unknown')

print(f"Predicted class (numerical): {predicted_label}")
print(f"Predicted class (full form): {predicted_full_form}")





1/1 [==============================] - 1s 949ms/step
Predicted class (numerical): akiec
Predicted class (full form): atinic keratoses and intraepithelial carcinoma


In [29]:
print(label_encoder.classes_)


['akiec' 'bcc' 'bkl' 'df' 'mel' 'nv' 'vasc']
